# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark RDD - SOLUTION
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful

In [1]:
from pyspark import SparkContext, SparkConf
import numpy as np
import operator

In [2]:
conf=SparkConf().setAppName("Lab4-rdd").setMaster("local[*]")
sc = SparkContext(conf=conf)

Using PySpark and RDD's on the https://coding.csel.io machines is slow -- most of the code is executed in Python and this is much less efficient than the java-based code using the PySpark dataframes. Be patient and trying using `.cache()` to cache the output of joins. You may want to start with a reduced set of data before running the full task. You can use the `sample()` method to extract just a sample of the data or use 

These two RDD's are called "rawCitations" and "rawPatents" because you probably want to process them futher (e.g. convert them to integer types, etc). 

The `textFile` function returns data in strings. This should work fine for this lab.

Other methods you use might return data in type `Byte`. If you haven't used Python `Byte` types before, google it. You can convert a value of `x` type byte into e.g. a UTF8 string using `x.decode('uft-8')`. Alternatively, you can use the `open` method of the gzip library to read in all the lines as UTF-8 strings like this:
```
import gzip
with gzip.open('cite75_99.txt.gz', 'rt',encoding='utf-8') as f:
    rddCitations = sc.parallelize( f.readlines() )
```
This is less efficient than using `textFile` because `textFile` would use the underlying HDFS or other file system to read the file across all the worker nodes while the using `gzip.open()...readlines()` will read all the data in the frontend and then distribute it to all the worker nodes.

In [3]:
rddCitations = sc.textFile("cite75_99.txt.gz")
rddPatents = sc.textFile("apat63_99.txt.gz")

# rddCitations.sample(False, 0.05)
# rddPatents.sample(False, 0.05)


The data looks like the following.

In [4]:
rddCitations.take(5)

['"CITING","CITED"',
 '3858241,956203',
 '3858241,1324234',
 '3858241,3398406',
 '3858241,3557384']

In [5]:
rddPatents.take(5)

['"PATENT","GYEAR","GDATE","APPYEAR","COUNTRY","POSTATE","ASSIGNEE","ASSCODE","CLAIMS","NCLASS","CAT","SUBCAT","CMADE","CRECEIVE","RATIOCIT","GENERAL","ORIGINAL","FWDAPLAG","BCKGTLAG","SELFCTUB","SELFCTLB","SECDUPBD","SECDLWBD"',
 '3070801,1963,1096,,"BE","",,1,,269,6,69,,1,,0,,,,,,,',
 '3070802,1963,1096,,"US","TX",,1,,2,6,63,,0,,,,,,,,,',
 '3070803,1963,1096,,"US","IL",,1,,2,6,63,,9,,0.3704,,,,,,,',
 '3070804,1963,1096,,"US","OH",,1,,2,6,63,,3,,0.6667,,,,,,,']

In other words, they are a single string with multiple CSV's. You will need to convert these to (K,V) pairs, probably convert the keys to `int` and so on. You'll need to `filter` out the header string as well since there's no easy way to extract all the lines except the first.

In the below cell the patent and citation RDDs are updated by removing the first row which contents the column names. So they now just store the values. 

In [6]:
# filter out the headers in both 
patents_header = rddPatents.take(1)
citations_header = rddCitations.take(1)

rddPatents = rddPatents.filter(lambda x: x not in patents_header)
rddCitations = rddCitations.filter(lambda x: x not in citations_header)
rddPatents.cache()
rddCitations.cache()

PythonRDD[9] at RDD at PythonRDD.scala:53

Next we create 3 new RDDs as key value pairs from the list. The patents_state RDD consists of the patent number as key and the state as value. the patent number is converted to integer. The same is done on the citations rdd. citing_kv consists of citing patent as the key and cited patent as the value. cited_kv is the vice-versa with cited patent as the key and citing patent as the value. Both cited and citing patents are converted to integers so help in manipulating the output later on. 

In [7]:
patents_state = rddPatents.map(lambda x: (int(x.split(",")[0]), str(x.split(",")[5]))).cache()
citing_kv = rddCitations.map(lambda x: (int(x.split(",")[0]), int(x.split(",")[1]))).cache()
cited_kv = rddCitations.map(lambda x: (int(x.split(",")[1]), int(x.split(",")[0]))).cache()

Next we perform a join between cited_kv and patents_state to get the information of the state for the cited patent. The cited_states RDD consists of cited patent as the key and the value is a tuple of citing patent and the state. 

In [8]:
cited_states = cited_kv.join(patents_state).cache()

Next we update citing_states by changing the key as the citing patent and value as a tuple of cited patent and state. This is done so that a join can be done with patents_state to get the citing state. After the join with patents_state we filter out all the rows with empty state value and also match the cited and citing states. The final result with the matching states for cited and citing patents is stored in the rdd co_states. 

In [9]:
# making citing the key (cited, state) as the value then joining back with patents to get citing state
citing_states = cited_states.map(lambda x: (x[1][0], (x[0], x[1][1]))).join(patents_state)
co_states = citing_states.filter(lambda x: x[1][1] != '""' and x[1][0][1] != '""' and str(x[1][1]) == str(x[1][0][1])) # not empty and having the same co states 
co_states.cache()


PythonRDD[28] at RDD at PythonRDD.scala:53

now that we have the matching states for cited and citing patent numbers. we group by citing patent number and and count the length of the iterator. the co_states_count RDD is a list of tuples with patent in the first index and the co_state count as the value. 

In [10]:
co_states_counts = co_states.groupByKey().map(lambda x : (x[0], len(list(x[1]))))
co_states_counts.cache()

PythonRDD[29] at RDD at PythonRDD.scala:53

Next we update the rddPatents with its key containing the patent number and value as list of the remaining items and it is stored in patents_info. patents_info is then joined to co_states_count to combine the co_state count to the patent_info.  

In [11]:
patents_info = rddPatents.map(lambda x: (int(x.split(",")[0]), x.split(",")[1:])).cache() # extracts patent number as key and other columns as values

final_join = patents_info.join(co_states_counts).cache()

finally the output is updated to keep the value as a list and not a tuple having the list and count. the output rdd here will have patent as the key and value would be a list of that patent's info with the last element in the list being the co_state count.

In [13]:
output = final_join.map(lambda kv: (kv[0], kv[1][0] + [kv[1][1]])).cache() # merge costate count into the list

the output is then printed out in the descending order of co_state count.

In [15]:
%%time
for line in output.takeOrdered(13, key = lambda x: -x[1][22]):
    print(line)

(5959466, ['1999', '14515', '1997', '"US"', '"CA"', '5310', '2', '', '326', '4', '46', '159', '0', '1', '', '0.6186', '', '4.8868', '0.0455', '0.044', '', '', 125])
(5983822, ['1999', '14564', '1998', '"US"', '"TX"', '569900', '2', '', '114', '5', '55', '200', '0', '0.995', '', '0.7201', '', '12.45', '0', '0', '', '', 103])
(6008204, ['1999', '14606', '1998', '"US"', '"CA"', '749584', '2', '', '514', '3', '31', '121', '0', '1', '', '0.7415', '', '5', '0.0085', '0.0083', '', '', 100])
(5952345, ['1999', '14501', '1997', '"US"', '"CA"', '749584', '2', '', '514', '3', '31', '118', '0', '1', '', '0.7442', '', '5.1102', '0', '0', '', '', 98])
(5958954, ['1999', '14515', '1997', '"US"', '"CA"', '749584', '2', '', '514', '3', '31', '116', '0', '1', '', '0.7397', '', '5.181', '0', '0', '', '', 96])
(5998655, ['1999', '14585', '1998', '"US"', '"CA"', '', '1', '', '560', '1', '14', '114', '0', '1', '', '0.7387', '', '5.1667', '', '', '', '', 96])
(5936426, ['1999', '14466', '1997', '"US"', '"CA"